In [103]:
import os
import re
from groq import Groq
from time import sleep

In [2]:
def caminho_arquivos(diretorio):
    paths = []
    for nome_arquivo in os.listdir(diretorio):
        paths.append(os.path.join(diretorio, nome_arquivo))

    return paths

In [3]:
def ler_arquivos(diretorio):
    textos = []
    for nome_arquivo in os.listdir(diretorio):
        if nome_arquivo.endswith('.txt'):
            with open(os.path.join(diretorio, nome_arquivo), 'r', encoding='utf-8') as file:
                textos.append(file.read())
    return textos

In [4]:
client = Groq(
    api_key="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",
)

### Pré-Processamento dos Textos

In [5]:
def ler_arquivo(path):
    try:
        with open(path, 'r', encoding='utf-8') as arquivo:
            conteudo = arquivo.read()
    except UnicodeDecodeError:
        with open(path, 'r', encoding='latin-1') as arquivo:
            conteudo = arquivo.read()
    return conteudo

In [6]:
def dividir_texto(texto, tamanho_maximo=10000):
    partes = []
    while len(texto) > tamanho_maximo:
        limite = texto.rfind('.', 0, tamanho_maximo)
        if limite == -1:
            limite = tamanho_maximo
        partes.append(texto[:limite + 1])
        texto = texto[limite + 1:].strip()
    partes.append(texto)
    return partes

In [7]:
def pre_processamento(path):
    conteudo = ler_arquivo(path)
    
    partes = dividir_texto(conteudo)
    
    resultado_final = ""
    
    for parte in partes:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": (
                        "Você é um assistente que ajuda a pré-processar dados legislativos. "
                        "Por favor, extraia e organize as informações legislativas do seguinte texto. "
                        "Certifique-se de remover conteúdo irrelevante, normalizar o texto e estruturá-lo "
                        "em seções claras. Inclua apenas informações importantes e relevantes sobre as leis, "
                        "resoluções e normas mencionadas no seguinte texto (tudo o que é informação relevante deve ser escrito, não resuma informações de forma alguma, e corrija erros de escrita se necessário):\n\n"
                        f"{parte}"
                    )
                }
            ],
            model="llama3-8b-8192",
        )
        
        resultado_final += chat_completion.choices[0].message.content + "\n"
    
    with open(f'pre_processados\\{path}', 'w', encoding='utf-8') as file:
        file.write(resultado_final)

In [8]:
caminhos = caminho_arquivos("textos")

In [9]:
for path in caminhos:
    pre_processamento(path)
    print(path)
    sleep(30)

textos\decreto_1044_69_afecções_orgânicas.txt
textos\decreto_2494_1998_educação_a_distância.txt
textos\decreto_2561_1998.txt
textos\decreto_5.626_05_língua_brasileira_de_sinais.txt
textos\estatuto_da_ufam.txt
textos\estágio_de_graduação.txt
textos\lei_10.098_2000_libras.txt
textos\lei_9.394_1996_diret_bases_educ_nacional.txt
textos\lei_carteira_estudantil.txt
textos\lei_nº_10.436_2002.txt
textos\lei_nº_12.089_2009.txt
textos\oficio_mec_nomclat_cursos.txt
textos\portaria_1.632_2006_pet.txt
textos\portaria_3.385_2005_mec.txt
textos\portaria_301_1998.txt
textos\portaria_no_1.095_de_25_de_outubro_de_2018_imprensa_nacional.txt
textos\port_048_2019_heteroidentificação.txt
textos\port_04_2002_prazo_ret_documentos.txt
textos\port_05_2020_aproveit_disci_do_pibid.txt
textos\port_2304_95_nada_consta.txt
textos\port_53_plano_de_ação_cursos_2018.txt
textos\port_proeg_propesp_001_2004_programa_mobilidade_.txt
textos\regimento_ufam.txt
textos\resolução_009_1999_consuni.txt
textos\resolução_010_2004_c

### Cálculo de quantas perguntas serão geradas por cada texto

In [16]:
diretorio_arquivos = 'pre_processados\\textos'
textos_legislacao = ler_arquivos(diretorio_arquivos)

In [17]:
base_de_dados = []
total_perguntas = 1000
texto_tamanho = [len(texto) for texto in textos_legislacao]

In [18]:
soma_tamanhos = sum(texto_tamanho)
proporcao_perguntas = [int((tamanho / soma_tamanhos) * total_perguntas) for tamanho in texto_tamanho]

### Geração das Instruções

In [26]:
def gerar_pergunta_resposta(texto_legislacao):
    prompt = (
        "Baseado na seguinte legislação:\n\n"
        f"{texto_legislacao}\n\n"
        "Gere uma pergunta e uma resposta que sejam coerentes e relevantes. "
        "Certifique-se de que a pergunta seja clara e a resposta forneça uma explicação precisa e completa. "
        "Use apenas um '\\n' para separar a pergunta e a resposta na sua resposta, e não inclua nenhum texto adicional. "
        "A estrutura deve ser:\n"
        "Pergunta\nResposta"
    )
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        model="llama3-8b-8192",
    )

    resposta_modelo = chat_completion.choices[0].message.content.strip()

    return resposta_modelo

In [41]:
def gerar_perguntas_respostas_para_texto_longo(texto, limite_caracteres=10000):
    perguntas_respostas = []
    
    partes_texto = dividir_texto(texto, limite_caracteres)
    
    for parte in partes_texto:
        try:
            resultado = gerar_pergunta_resposta(parte)
            perguntas_respostas.append(resultado)
        except Exception as e:
            print(f"Erro ao gerar pergunta e resposta para uma parte do texto: {e}")
    
    return perguntas_respostas

In [33]:
base_de_dados = []

In [34]:
i = 0
for texto, num_instrucoes in zip(textos_legislacao, proporcao_perguntas):
    for _ in range(num_instrucoes):
        if len(texto) < 20000:
            pergunta_resposta = gerar_pergunta_resposta(texto)
            base_de_dados.append(pergunta_resposta)
            sleep(20)
        else:
            print(f"Falta processar {i}")
        i += 1

Falta processar 113
Falta processar 114
Falta processar 115
Falta processar 116
Falta processar 117
Falta processar 118
Falta processar 119
Falta processar 120
Falta processar 121
Falta processar 122
Falta processar 123
Falta processar 124
Falta processar 125
Falta processar 126
Falta processar 127
Falta processar 128
Falta processar 129
Falta processar 130
Falta processar 131
Falta processar 132
Falta processar 133
Falta processar 134
Falta processar 135
Falta processar 136
Falta processar 137
Falta processar 138
Falta processar 139
Falta processar 140
Falta processar 141
Falta processar 142
Falta processar 143
Falta processar 144
Falta processar 145
Falta processar 146
Falta processar 147
Falta processar 148
Falta processar 149
Falta processar 150
Falta processar 151
Falta processar 152
Falta processar 153
Falta processar 154
Falta processar 155
Falta processar 156
Falta processar 157
Falta processar 158
Falta processar 159
Falta processar 160
Falta processar 161
Falta processar 162


In [ ]:
for texto, num_instrucoes in zip(textos_legislacao, proporcao_perguntas):
    for _ in range(num_instrucoes):
        if len(texto) >= 20000:
            pergunta_resposta = gerar_perguntas_respostas_para_texto_longo(texto)
            base_de_dados.append(pergunta_resposta)
            sleep(30)

In [44]:
len(base_de_dados)

1157

In [91]:
base_de_dados[5]

'Pergunta: Quais são as exigências para o credenciamento de instituições que oferecem cursos de educação a distância?\n\nResposta: A autorização, o reconhecimento de cursos e o credenciamento de instituições que oferecem cursos a distância devem observar o que estabelece este Decreto e a legislação específica. O credenciamento é limitado a cinco anos e pode ser renovado após avaliação. A avaliação seguirá procedimentos, critérios e indicadores de qualidade definidos em ato próprio expedido pelo Ministro de Estado da Educação e do Desporto.'

In [92]:
def isString(item):
    if isinstance(item, str): return [item]
    else: return item

In [115]:
def separar_perguntas_respostas(item):
    perguntas_respostas = []
    
    matches = re.findall(r'Pergunta:\s*(.*?)\n\nResposta:\s*(.*)', item, re.DOTALL)
    
    if not matches:
        parts = item.split('\n\n')
        if len(parts) >= 2:
            pergunta = parts[0].strip()
            resposta = parts[1].strip()
            perguntas_respostas.append({
                'prompt': pergunta,
                'completion': resposta
            })
    else:
        for pergunta, resposta in matches:
            pergunta = pergunta.strip()
            resposta = resposta.strip()
            perguntas_respostas.append({
                'prompt': pergunta,
                'completion': resposta
            })
    
    return perguntas_respostas

In [116]:
dataset = []

for item in base_de_dados:
    if isinstance(item, str):
        resultado = separar_perguntas_respostas(item)
        dataset.extend(resultado)

In [1]:
import csv
import json

In [ ]:
with open('perguntas_respostas.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['text', 'completion']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for entry in dataset:
        json_entry = json.dumps(entry, ensure_ascii=False)
        csvfile.write(f"{json_entry}\n")